## CS431/631 Big Data Infrastructure
### Winter 2023 - Assignment 1
---

**Please edit this (text) cell to provide your name and UW student ID number!**
* **Name:** _Daolin An_
* **ID:** _20885166_

---
#### Overview
For this assignment, you will be using Python to analyze the [pointwise mutual information (PMI)](http://en.wikipedia.org/wiki/Pointwise_mutual_information) of tokens in the text of Shakespeare's plays.    For this assignment, you will need the same text file (`Shakespeare.txt`) that you used for Assignment 0.   You will also need the Python tokenizer module, `simple_tokenize.py`.

If two events $x$ and $y$ are independent, their PMI will be zero.   A positive PMI indicates that $x$ and $y$ are more likely to co-occur than they would be if they were independent.   Similarly, a negative PMI indicates that $x$ and $y$ are less likely to co-occur.   The PMI of events $x$ and $y$ is given by
\begin{equation*}
PMI(x,y) = \log\frac{p(x,y)}{p(x)p(y)}
\end{equation*}
where $p(x)$ and $p(y)$ are the probabilities of occurrence of events $x$ and $y$, and $p(x,y)$ is the probability of co-occurrence of $x$ and $y$.

For this assignment, the "events" that we are interested in are occurrences of tokens on lines of text in the input file.   For example, one event
might represent the occurence of the token "fire" a line of text, and another might represent the occurrence of the token "peace".   In that case, $p(fire)$ represents the probability that "fire" will occur on a line of text, and $p(fire,peace)$ represents the probability that *both* "fire" and "peace" will occur on the *same* line.   For the purposes of these PMI computations, it does not matter how many times a given token occures on a single line.   Either a line contains a particular token (at least once), or it does not.   For example, consider this line of text:

> three three three, said thrice

For this line, the following token-pair events have occurred:
- (three, said)
- (three, thrice)
- (said, three)
- (said, thrice)
- (thrice, three)
- (thrice, said)

Note that we are not interested in "reflexive" pairs, such as (thrice,thrice).

In addition to the probabilities of events, we will also be interested in the absolute *number* of occurences of particular events, e.g., the number of lines in which "fire" occurs.   We will use $n(x)$ to represent the these numbers.

Your main task for this assignment is to write Python code to analyze the PMI of tokens from Shakespeare's plays.    Based this analysis, we want to be able to answer two types of queries:

* Two-Token Queries: Given a pair of tokens, $x$ and $y$, report the number of lines on which that pair co-occurs ($n(x,y)$) as well as $PMI(x,y)$.
* One-Token Queries: Given a single token, $x$, report the number of lines on which that token occurs ($n(x)$).   In addition, report the five tokens that have the largest PMI with respect to $x$ (and their PMIs).   That is, report the five $y$'s for which $PMI(x,y)$ is largest.

To avoid reporting spurious results for the one-token queries, we are only interested in token pairs that co-occur a sufficient number of times.   Therefore, we will use a *threshold* parameter for one-token queries.   A one-token query should only report pairs of tokens that co-occur at least *threshold* times in the input.   For example, given the threshold 12, a one-token query for "fire" the should report the five tokens that have the largest PMI (with respect to "fire") among all tokens that co-occur with "fire" on at least 12 lines.   If there are fewer than five such tokens, report fewer than five.



Run the next block to download the text file (`Shakespeare.txt`) and the Python tokenizer module (`simple_tokenize.py`).

In [1]:
!wget -q https://student.cs.uwaterloo.ca/~cs451/content/cs431/Shakespeare.txt
!wget -q https://student.cs.uwaterloo.ca/~cs451/content/cs431/simple_tokenize.py

---
#### Question 1  (2/10 marks):

Before writing code to handle the PMI queries, start writing some code to answer some simpler questions that give an
idea of how big the PMI analysis problem will be.   The box below contains some starter code that reads in the 'Shakespeare.txt' file and tokenizes it one line at time.   (This is the same code you started with for Assignment 0.)  Extend this code to determine (a) the number of *distinct* tokens that exist in 'Shakespeare.txt', and (b) the number of 
*distinct* token pairs that exist in 'Shakespeare.txt'.  For the purposes of this question, consider the token pair $x,y$ to be distinct from the pair $y,x$, i.e., count them both.   Ignore token pairs of the form $x,x$. Note that we're considering things one line at a time, so the number of unique pairs is NOT (n)(n-1) where n is the number of unique tokens! If a line is "this is fine" then there are 6 distinct pairs on that line.

As a sanity check, when rounded you should be getting ~26K distinct tokens and ~2M distinct pairs. (Please remember I ROUNDED so e..g if you say "I got 1.9M is that OK" then I will worry about you) 

In [2]:
# this imports the SimpleTokenize function from the simple_tokenize.py file that you uploaded
from simple_tokenize import simple_tokenize
# permutations() function from itertools is used in this method
from itertools import permutations

# Now, let's tokenize Shakespeare's plays
with open('Shakespeare.txt') as f:
    # dictionary variable to store distinct tokens
    dict_token = {}
    # dictionary variable to store distinct token pairs
    dict_token_pair = {}
    for line in f:
        # tokenize, one line at a time
        t = simple_tokenize(line)
        # set variable to store tokens that has already been encountered in current line
        occurred_token = set()
        # list variable to store distinct tokens in current line
        distinct_token = []
        for token in t:
            # add current token to dict_token if it is not in dict_token
            #   or increase the value associate with the token key by one
            dict_token[token] = dict_token.get(token, 0) + 1
            # if current token is distinct from former tokens in the line,
            #   add it to the distinct_token list and occurred_token set
            if token not in occurred_token:
                distinct_token.append(token)
                occurred_token.add(token)
        # get all distinct token pairs in current line
        perm = permutations(distinct_token, 2)
        for ele in perm:
            # if current token pair has not been in dict_token_pair yet,
            #   add the pair to it. or increase the value associate with
            #   the token pair key by one
            dict_token_pair[ele] = dict_token_pair.get(ele, 0) + 1
    print("Number of distinct tokens are:", len(dict_token))
    print("Number of distinct tokens pairs are:", len(dict_token_pair))

    
# extend this code to answer Question 1.
# when your code is executed, it should print the number of distinct tokens and the number of distinct token pairs

Number of distinct tokens are: 25975
Number of distinct tokens pairs are: 1969760
122458


---

#### Question 2 (6/10 marks):
Next, write Python code to answer the one-token and two-token queries described above, for 'Shakespeare.txt'.   The code cell below contains some starter code that implements a simple text-based query interface.  It allows a user to ask a series of one-token or two-token queries.   Try running the starter code to get a sense of how the interface behaves.    

Your task is to write code to read and tokenize 'Shakespeare.txt', record information that will allow both types of PMI queries to be answered, and then answer queries that are posed through the query interface.  Make sure that your code is well commented, so that it will be clear to the markers.

If you cannot get answers to both types of queries working, try to get at least one type working, for partial credit.

Please appropriately handle cases where the token being queried isn't in the file. Print an appropriate error message and continue reading input.
(We won't mark the text of your error messages, but it's bad practice to crash if someone has a typo)

Sanity checks: 

```
Input 1 or 2 space-separated tokens (return to quit): the end
n(the,end) = 157
PMI(the,end) = 0.3505058356267105
```

If you got n=147 then you didn't use `simple_tokenize()`, you used `split()` and were thrown off by puntuation and capitlization.

(If you haven't been told this before, floating point numbers are inexact. You do not need to get exactly those digits, but should match at least 4 of them! Make sure you used log base 10)

If your PMI is 2.9ish then you probably are counting how many times each token occurrs rather than how many LINES contain that token. 

Because of the phrase "at rope's end", you should also expect that "rope's" doesn't really show up outside this phrase, so it will have a high PMI with "end".

```
n(end,rope's) = 5, PMI(end,rope's) = 2.5ish
```


In [ ]:
# this imports the SimpleTokenize function from the simple_tokenize.py file that you uploaded
from simple_tokenize import simple_tokenize
# the log function for computing PMI
# for the sake of consistency across solutions, please use log base 10
from math import log

###################################################################################################################
#  replace this with your PMI analysis code, so that you can support the user interface below
#  it should read and tokenize Shakespeare.txt, and store enough information in Python data structures
#  to allow you to answer the PMI queries below
###################################################################################################################

# Function to calculate pmi given p(x), p(y), p(x and y)
def PMI(p_x, p_y, p_xy):
    pmi = log(p_xy/(p_x * p_y), 10)
    return pmi

# Process the given file to get all the information needed to calculate pmi
def processingFile(file = 'Shakespeare.txt'):
    distinct_tokens = set() # to store distinct tokens
    info = {} ## dict{token1: [occurrence, dict{token2: coocurrence}]}
              ## occurrence => num of lines token1 occurs in the file
              ## cooccurrence => num of lines token1 and token2 cooccur in the file
    num_lines = 0 # store number of lines in the file
    with open(file) as f:
        for line in f:
            tokens = set(simple_tokenize(line))
            num_lines += 1
            for token in tokens:
                info[token] = info.get(token, [0, {}])
                info[token][0] += 1
                rest_tokens = tokens - {token}
                for t in rest_tokens:
                    dict_dict = info[token][1]
                    if t not in dict_dict:
                        dict_dict[t] = 0
                    dict_dict[t] += 1
    return [num_lines, info]

def calculate(num_lines, info, x, y = '', threshold = -1):
    ## input value checking
    if threshold == -1:
        if x not in info and y not in info:
            print('Token ' + x + ' and ' + y + ' are not in the file!')
            return None
        elif x not in info:
            print('Token ' + x + ' is not in the file!')
            return None
        elif y not in info:
            print('Token ' + y + ' is not in the file!')
            return None
    else:
        if x not in info:
            print('Token ' + x + ' is not in the file!')
            return None
    ## initialization
    p_x = info[x][0] / num_lines
    p_y = 0
    p_xy = 0
    ## two token case
    if threshold == -1:
        p_y = info[y][0] / num_lines
        num_xy = info[x][1][y]
        p_xy = num_xy / num_lines
        return [num_xy, PMI(p_x, p_y, p_xy)]
    ## one-token case
    info_keys = list(info.keys())
    dict_pmi = {} # store the calculated pmi
    for key in info_keys:
        if key == x:
            continue
        cooccurrence = -1
        if key in info[x][1]:
            cooccurrence = info[x][1][key]
        ## calculate the pmi of token pair (x, y) if they pass the threshold
        if cooccurrence >= threshold:
            p_y_key = info[key][0] / num_lines
            p_xy_key = cooccurrence / num_lines
            pmi = PMI(p_x, p_y_key, p_xy_key)
            dict_pmi[key] = pmi
    ## sort the dict_pmi to get the first five elements with largest pmi
    sorted_dict_pmi = sorted(dict_pmi.items(), key = lambda x: x[1], reverse = True)
    length = 5 if len(sorted_dict_pmi) > 5 else len(sorted_dict_pmi)
    if length == 0:
        print('No token pairs pass the threshold!' )
    high_pmi_tokens = sorted_dict_pmi[:length]
    return high_pmi_tokens

result = processingFile()
num_lines = result[0]
info = result[1]

###################################################################################################################
#  the user interface below defines the types of PMI queries that users can ask
#  you will need to modify it - where indicated - to access the results of your PMI analysis (above)
#  so that the queries can be answered
###################################################################################################################

while True:
    q = input("Input 1 or 2 space-separated tokens (return to quit): ")
    if len(q) == 0:
        break
    q_tokens = simple_tokenize(q)
    if len(q_tokens) == 1:
        threshold = 0
        while threshold <= 0:
            try:
                threshold = int(input("Input a positive integer frequency threshold: "))
            except ValueError:
                print("Threshold must be a positive integer!")
                continue
        # Put code here to answer a One-Token Query with token q_tokens[0] and the specified threshold,
        # and output the result.
        # The print() statements below exist to show you the desired output format.
        # Replace them with your own output code, which should produce results in a similar format.
        high_pmi_tokens = calculate(num_lines, info, x = q, threshold = threshold)
        if high_pmi_tokens == None:
            continue
        num_x = info[q][0] ## occurrence of the given token
        num_tokens = len(high_pmi_tokens)
        print("  n({0}) = {1}".format(q_tokens[0], num_x))
        print("  high PMI tokens with respect to {0} (threshold: {1}):".format(q_tokens[0],threshold))
        for i in range(num_tokens):
            name_token = high_pmi_tokens[i][0]
            num_cooccurrence = info[name_token][1][q_tokens[0]]
            pmi = high_pmi_tokens[i][1]
            print("    n({0},{1}) = {2},  PMI({0},{1}) = {3}".format(q_tokens[0], name_token, num_cooccurrence, pmi))
    elif len(q_tokens) == 2:
        # Put code here to answer a Two-Token Query with tokens q_tokens[0] and q_tokens[1]
        # As was the case for the One-Token query, the print statements below show the desired output format
        # Replace them with your own output code
        analysis_two_token = calculate(num_lines, info, x = q_tokens[0], y = q_tokens[1])
        if analysis_two_token == None:
            continue
        num_token = analysis_two_token[0]
        pmi_token = analysis_two_token[1]
        print("  n({0},{1}) = {2}".format(q_tokens[0],q_tokens[1], num_token))
        print("  PMI({0},{1}) = {2}".format(q_tokens[0],q_tokens[1], pmi_token))
    else:
        print("Input must consist of 1 or 2 space-separated tokens!")
        

Input 1 or 2 space-separated tokens (return to quit):  as
Input a positive integer frequency threshold:  10


  n(as) = 4917
  high PMI tokens with respect to as (threshold: 10):
    n(as,lief) = 16,  PMI(as,lief) = 1.3699580151911006
    n(as,disguised) = 11,  PMI(as,disguised) = 1.0759518030540822
    n(as,snow) = 14,  PMI(as,snow) = 0.9513503825651887
    n(as,easy) = 20,  PMI(as,easy) = 0.8844035929345756
    n(as,twere) = 36,  PMI(as,twere) = 0.8626843432413394


---

#### Question 3 (2/10 marks):

Suppose that you try to run your PMI analysis on larger files:  say, 10 times, or 100 times, or 1000 times larger than 'Shakespeare.txt'.    As the input file grows larger, what will happen to the execution of your program?   Will it get slower?   How much slower?   Will it eventually fail to run?   If so, why?

Consider two seperate parts: The initial processing of the file, and then the time required to run the queries themselves.
(If you did not do any initial processing and reloaded everything for each query, then please go fix that, that's not okay.)


In the cell below, briefly (one or two paragraphs), discuss what will happen if the input to your analysis grows.  We're not looking for an exact or empirical analysis of the behaviour of your program as a function of input size.  Rather, we are looking for a discussion of trends and limits.

(Tip since we probably haven't got that far in lectures yet: Vocabulary size for a document actually grows unbounded, proportional to the log of the document size!)

#### Answer to Question 3:

I split the PMI analysis into two part. One for processing the file, and the other for the pmi calculation query. The worst case run time for my processing function is O(n^2), the worst case run time for two-token pmi calculation is O(nlogn), and the worst case run time for one-token pmi calculation is O(1).

The file-processing function only needs to be run once, but it may be much slower to run the input file becomes 10 times, 100 times larger since I used a permutation-like method to get all the information needed for the pmi calculation.

The run time of two-token case query would still be fast since it only extract two elements from a dictionary and then perform calculation.
The run time of one-token case query may be a little bit slower since it involves sorting a dictionary.

---
Don't forget to save your workbook!   (It's a good idea to do this regularly, while you are working.)   When you are finished and you are ready to submit your assignment, download your notebook file (.ipynb) from the hub to your machine, and then follow the submission instructions in the assignment.